# Functions for Austria and Germany
We compute the UI benefit duration and non-employment duration.

The Poisson process is nonhomogeneous in this non-stationary environment.

## 1) UI benefit duration
### a) Realised dynamics 
The probability density function (pdf) of UI benefit duration $B$, conditional on age at the start of the unemployment spell $a$, is
\begin{align}
& f_{UI}(B|a)=\left\{ 
\begin{array}{ll}
\exp \left[-\lambda_0 B -\int_{0}^{B}\mathcal J(S_2(a+t))dt\right].(\lambda_0+\mathcal J(S_2(a+B))) & \text{ if (1)} \\
\exp \left[-\lambda_0 B  -\int_{0}^{A-a}\mathcal J(S_2(a+t))dt-\mathcal J(\Sigma_2)(a+B-A)\right].(\lambda_{max}+\mu(\Sigma_2)) & \text{ if (2)} \\
\exp \left[ -(\lambda _1+\mathcal J(\Sigma_3))B\right].(\lambda_1+\mathcal J(\Sigma_3)) & \text{ if (3)}
\end{array}
\right. 
.
\end{align}
with 
\begin{align}
&  \text{(1) $a<A$ and $a+B\leq A$}, \\
& \text{(2) $a<A$ and $a+B>A$}, \\
& \text{(3) $a\geq A$}.
\end{align}
We obtain the expectation with $\mathbb{E}(B|a)=\int_0^{\infty} Bf_{UI}(B|a)dB$.
For $a\geq A$, $\mathbb{E}(B|a)=\frac{1}{\lambda_1+\mathcal J(\Sigma_3)}$.

In [ ]:
function fUI_case1(B::Float64, a::Float64, V::Surplus, C::Union{Country2,Country2A}) ## for a<A and a+B<A
    smallintegral = quadgk(t->  JFR(V.S(a+t)[2],C) , 0, B)[1]
    expo = exp( -C.λ0*B -smallintegral)
    out = (C.λ0 + JFR(V.S(a+B)[2],C)) * expo
    return(out)
end

function fUI_case2(B::Float64, a::Float64, V::Surplus, C::Country2) ## for a<A and a+B>A
    smallintegral = quadgk(t-> JFR(V.S(a+t)[2],C) , 0, C.A-a)[1]
    expo = exp( -C.λ0*B -smallintegral - (a+B-C.A)* JFR(V.Σ[2],C))
    out = (C.λ0 + JFR(V.Σ[2],C)) * expo
    return(out)
end

function fUI_case3(B::Float64, V::Surplus, C::Country2) ## for a>A
    expo = exp( -(C.λ1 + JFR(V.Σ[3],C)) * B )
    out = (C.λ1 + JFR(V.Σ[3],C)) * expo
    return(out)
end

function fUI(B::Float64, a::Float64, V::Surplus, C::Country2)
    if a+B<C.A
        return(fUI_case1(B, a, V, C))
    elseif a < C.A
        return(fUI_case2(B, a, V, C))
    else
        return(fUI_case3(B, V, C))
    end
end

function expectedB(a::Float64, V::Surplus, C::Country2)
    if a>=C.A
        return 1/ (C.λ1+JFR(V.Σ[3],C))
    else
        part1 = quadgk(B-> B * fUI_case1(B,a,V,C), 0,C.A-a,atol=5e-3)[1]
        part2 = quadgk(B-> B * fUI_case2(B,a,V,C), C.A-a,Inf, atol=5e-3)[1]
        ## we specifiy an error of 1e-2 (month)
        return part1+part2
    end
end

For the life-cycle model, we have:
\begin{align}
& f_{UI}(B|a)=\left\{ 
\begin{array}{ll}
\exp \left[-\lambda_0 B -\int_{0}^{B}\mathcal J(S_2(a+t))dt\right].(\lambda_0+\mathcal J(S_2(a+B))) & \text{ if (1)} \\
\exp \left[-\lambda_0 B  -\int_{0}^{A-a}\mathcal J(S_2(a+t))dt-\int_{A-a}^{B}\mathcal J(\Sigma_2(a+t))dt\right].(\lambda_0+\mu(\Sigma_2(a+B))) & \text{ if (2)} \\
\exp \left[ -\lambda_1 B - \int_{0}^{B}\mathcal J(\Sigma_3(a+t))dt\right].(\lambda_1+\mathcal J(\Sigma_3(a+B))) & \text{ if (3)}
\end{array}
\right. 
.
\end{align}
with 
\begin{align}
&  \text{(1) $a<A$ and $a+B\leq A$}, \\
& \text{(2) $a<A$ and $a+B>A$}, \\
& \text{(3) $a\geq A$}.
\end{align}
We obtain the expectation with $\mathbb{E}(B|a)=\int_0^{A_{ret}-a} Bf_{UI}(B|a)dB$.

In [ ]:
## fUI_case1 is the same for Country2A

function fUI_case2(B::Float64, a::Float64, V::Surplus, C::Country2A) ## for a<A and a+B<A
    smallintegral1 = quadgk(t->  JFR(V.S(a+t)[2],C) , 0, C.A-a)[1]
    smallintegral2 = quadgk(t->  JFR(V.Σ(a+t)[2],C) , C.A-a, B)[1]
    expo = exp( -C.λ0*B - smallintegral1 - smallintegral2)
    out = (C.λ0 + JFR(V.Σ(a+B)[2],C)) * expo
    return(out)
end

function fUI_case3(B::Float64, a::Float64, V::Surplus, C::Country2A) ## for a>A
    smallintegral = quadgk(t->  JFR(V.Σ(a+t)[3],C) , 0, B)[1]
    expo = exp( -C.λ1*B -smallintegral)
    out = (C.λ1 + JFR(V.Σ(a+B)[3],C)) * expo
    return(out)
end

function fUI(B::Float64, a::Float64, V::Surplus, C::Country2A)
    if a+B<C.A
        return(fUI_case1(B, a, V, C))
    elseif a < C.A
        return(fUI_case2(B, a, V, C))
    else
        return(fUI_case3(B, a, V, C))
    end
end

function expectedB(a::Float64, V::Surplus, C::Country2A)
    B_inf = C.Aret-a ##maximum value of B given retirement age
    if a>=C.A
        return quadgk(B-> B * fUI_case3(B,a,V,C), 0, B_inf, atol=5e-3)[1]
    else
        part1 = quadgk(B-> B * fUI_case1(B,a,V,C), 0,C.A-a, atol=5e-3)[1]
        part2 = quadgk(B-> B * fUI_case2(B,a,V,C), C.A-a, B_inf, atol=5e-3)[1]
        ## we specify an error of 1e-2 (month)
        return part1+part2
    end
end

### b) Counterfactuals
In a world with unconditional policy, the pdf of $B$ is exactly the pdf in the first case:
$$f_{UI}^{with}(B|a)=\exp \left[ -(\lambda_1+\mathcal J(\Sigma_3))B\right].(\lambda_1+\mathcal J(\Sigma_3))$$
with no life-cycle components.
This gives a simple formula of the expected duration:
$\mathbb{E}^{with}(B|a)=\frac{1}{\lambda_1+\mathcal J(\Sigma_3)}$.

In the life-cycle version,
$$f_{UI}^{with}(B|a)=\exp \left[ -\lambda_1 B - \int_{0}^{B}\mathcal J(\Sigma_3(a+t))dt\right].(\lambda_1+\mathcal J(\Sigma_3(a+B)))$$
and we still need to integrate: $\mathbb{E}^{with}(B|a)=\int_0^{A_{ret}-a} Bf_{UI}^{with}(B|a)dB$.

In [ ]:
expectedB_with(a::Float64, V::Surplus, C::Country2) = 1/ (C.λ1+JFR(V.Σ[3],C))

fUI_with(B::Float64, a::Float64, V::Surplus, C::Country2A) = fUI_case3(B,a,V,C)

function expectedB_with(a::Float64, V::Surplus, C::Country2A)
    B_inf = C.Aret-a ##maximum value of B given retirement age
    return quadgk(B-> B * fUI_with(B,a,V,C), 0, B_inf, atol=5e-3)[1]
end

In a world without the policy, the pdf of $B$ is
    $$f_{UI}^{without}(B|a)=\exp \left[ -(\lambda_0+\mathcal J(\Psi_2))B\right].(\lambda_0+\mathcal J(\Psi_2))$$
with no life-cycle components.
This gives a simple formula of the expected duration:
$\mathbb{E}^{without}(B|a)=\frac{1}{\lambda_0+\mathcal J(\Psi_2)}$.

I the life-cycle version,
$$f_{UI}^{without}(B|a)=\exp \left[ -\lambda_0 B - \int_{0}^{B}\mathcal J(\Psi_2(a+t))dt\right].(\lambda_0+\mathcal J(\Psi_2(a+B)))$$
and we still need to integrate: $\mathbb{E}^{without}(B|a)=\int_0^{A_{ret}-a} Bf_{UI}^{without}(B|a)dB$.

In [ ]:
expectedB_without(a::Float64, V::Surplus, C::Country2) = 1/ (C.λ0+JFR(V.Ψ[2],C))

function fUI_without(B::Float64, a::Float64, V::Surplus, C::Country2A)
    smallintegral = quadgk(t->  JFR(V.Ψ(a+t)[2],C) , 0, B)[1]
    expo = exp( -C.λ0*B -smallintegral)
    out = (C.λ0 + JFR(V.Ψ(a+B)[2],C)) * expo
    return out
end

function expectedB_without(a::Float64, V::Surplus, C::Country2A)
    B_inf = C.Aret-a ##maximum value of B given retirement age
    return quadgk(B-> B * fUI_without(B,a,V,C), 0, B_inf, atol=5e-3)[1]
end

## 2) Non-employment duration
### a) Realised dynamics
We now characterize the age-conditional pdf of non-employment duration $D$.


Case 1 for $a<A$ and $a+D\leq A$:
\begin{align}
f_{NE}(D|a)& =\underbrace{f_{UI}(D|a) . \frac{\mathcal J(S_2(a+D))}{\lambda_0 + \mathcal J(S_2(a+D))}}_{\text{term 1}} \notag \\
& +\mathcal J(S_1(a+D)) .\int_{0}^{D} \underbrace{f_{UI}(B|a) . \frac{\lambda_0}{\lambda_0+\mathcal J(S_2(a+B))}.\exp \left[
-\int_{B}^{D} \mathcal J(S_1(a+t))dt\right] }_{\text{integrand_case1}} dB
\end{align}

Case 2 for $a<A$ and $a+D> A$:
\begin{align}
f_{NE}(D|a)& =\underbrace{f_{UI}(D|a) . \frac{\mathcal J(\Sigma_2)}{\lambda_0 + \mathcal J(\Sigma_2)}}_{\text{term 1}} \notag \\
& +\exp\left[ -\mathcal J(\Sigma_1). (a+D-A)\right]  .\mathcal J(\Sigma_1) . \int_{0}^{A-a} \underbrace{f_{UI}(B|a) . \frac{\lambda_0 }{\lambda_0 + \mathcal J(S_2 (a+B))}.\exp \left[-\int_{B}^{A-a}\mathcal J(S_1(a+t))dt \right]}_{\text{integrand_case2A}} dB\notag \\
& +\frac{\lambda_0 }{\lambda_0 + \mathcal J(\Sigma_2)} .\mathcal J(\Sigma_1) . \int_{A-a}^{D} \underbrace{f_{UI}(B|a) . \exp \left[
-\mathcal J(\Sigma_1).(D-B)\right]}_{\text{integrand_case2B}} dB.
\end{align}

Case 3 $a\geq A$:
\begin{align}
f_{NE}(D|a)& =f_{UI}(D|a) . \frac{\mathcal J(\Sigma_3)}{\lambda_1 + \mathcal J(\Sigma_3)} +\mathcal J(\Sigma_1) .\int_{0}^{D} f_{UI}(B|a) . \frac{\lambda_1}{\lambda_1 + \mathcal J(\Sigma_3)}.\exp \left[ -\mathcal J(\Sigma_1)(D-B)\right]  dB\\
 &=\frac{(\lambda_1 +\mathcal J(\Sigma_3)).(\mathcal J(\Sigma_1)-\mathcal J(\Sigma_3))}
{\mathcal J(\Sigma_1)-(\lambda_1 +\mathcal J(\Sigma_3))}
e^{-(\lambda_1 +\mathcal J(\Sigma_3))D}  \notag \\
& -\frac{\lambda_1 \mathcal J(\Sigma_1)}{\mathcal J(\Sigma_1)-(\lambda_1 +\mathcal J(\Sigma_3))}e^{-\mathcal J(\Sigma_1)D}
\end{align}

The expectation is $\mathbb{E}(D|a)=\int_0^{\infty}Df_{NE}(D|a)dD.$
For $a>A$, we have a simple expression:
\begin{align}
\mathbb{E}(D|a)& =\frac{\mathcal J(\Sigma_1)-\mathcal J(\Sigma_3)}{(\lambda_1 +\mathcal J(\Sigma_3))(\mathcal J(\Sigma_1)-\lambda_1 -\mathcal J(\Sigma_3))}  -\frac{\lambda_1}{\mathcal J(\Sigma_1).(\mathcal J(\Sigma_1)-\lambda_1 -\mathcal J(\Sigma_3))}.
\end{align}


In [ ]:
### case 1
function integrand_fNE_case1(B::Float64, D::Float64, a::Float64, V::Surplus, C::Union{Country2,Country2A})
    comp1 = fUI_case1(B,a,V,C)  ## fUI_case1 because a<a+B<A, because a<a+D<A
    comp2 = C.λ0 / (C.λ0+JFR(V.S(a+B)[2],C))
    integ3 = quadgk(s-> JFR(V.S(a+s)[1],C) , B, D)[1]
    return(comp1 * comp2 * exp(-integ3))
end

function fNE_case1(D::Float64, a::Float64, V::Surplus, C::Union{Country2,Country2A}) 
    term1 = fUI_case1(D,a,V,C) * JFR(V.S(a+D)[2],C) / (C.λ0+JFR(V.S(a+D)[2],C)) ## fUI_case1 because a<a+D<A
    term2 = JFR(V.S(a+D)[1],C) * quadgk(B->integrand_fNE_case1(B,D,a,V,C), 0,D)[1] 
    return(term1+term2)
end

### case 2
function integrand_fNE_case2A(B::Float64, D::Float64, a::Float64, V::Surplus, C::Country2)
    term1 = fUI_case1(B,a,V,C) ## fUI_case1 because a+B<A in case 2A
    term2 = C.λ0 / (C.λ0+JFR(V.S(a+B)[2],C)) 
    integ3 = quadgk(s-> JFR(V.S(a+s)[1],C) , B, C.A-a)[1]
    return term1*term2*exp(-integ3)
end

function integrand_fNE_case2B(B::Float64, D::Float64, a::Float64, V::Surplus, C::Country2)
    return fUI_case2(B,a,V,C) * exp(-JFR(V.Σ[1],C) * (D-B)) ## fUI_case2 because A<a+B in case 2B
end 

function fNE_case2(D::Float64, a::Float64, V::Surplus, C::Country2) 
    term1 = fUI_case2(D,a,V,C) * JFR(V.Σ[2],C) / (C.λ0+JFR(V.Σ[2],C)) ## fUI_case2 because a<A<a+D
    term2A = exp( -JFR(V.Σ[1],C) * (a+D-C.A) ) * JFR(V.Σ[1],C) * quadgk(B->integrand_fNE_case2A(B,D,a,V,C), 0,C.A-a)[1]
    term2B = C.λ0 / (C.λ0+JFR(V.Σ[2],C)) * JFR(V.Σ[1],C) * 
            quadgk(B->integrand_fNE_case2B(B,D,a,V,C), C.A-a, D)[1]
    return term1+term2A+term2B
end
    

function expectedD(a::Float64, V::Surplus, C::Country2)
    if a>=C.A
        mu1 = JFR(V.Σ[1],C)
        mu3 = JFR(V.Σ[3],C) # with lambdamin
        lam = C.λ1
        return((mu1-mu3)/((lam+mu3)*(mu1-lam-mu3)) - lam/(mu1*(mu1-lam-mu3)))
    else
        return( quadgk(D-> D * fNE_case1(D,a,V,C), 0,C.A-a,atol=5e-3)[1]+
            quadgk(D-> D * fNE_case2(D,a,V,C), C.A-a,Inf, atol=5e-3)[1] )
        ## we specify an error of 1e-2 (month)
    end
end



### Codes below are not necessary
function fNE_case3(D::Float64, V::Surplus, C::Country2) 
    mu1 = JFR(V.Σ[1],C)
    mu3 = JFR(V.Σ[3],C) # with lambdamin
    lam = C.λ1
    term1 = (lam+mu3) * (mu1-mu3) / (mu1-lam-mu3) * exp(-(lam+mu3)*D)
    term2 = - lam*mu1 / (mu1-lam-mu3) * exp(-mu1 * D)
    return term1 + term2
end



### we code fNE, but this is not necessary
function fNE(D::Float64, a::Float64, V::Surplus, C::Country2)
    if a+D<C.A
        return(fNE_case1(D, a, V, C))
    elseif a < C.A
        return(fNE_case2(D, a, V, C))
    else
        return(fNE_case3(D, V, C)) 
    end
end

In [ ]:
### here we code a slower version of fNE but more reliable
function routine_fNE_mu1(a::Float64, V::Surplus, C::Country2)
    if a<C.A
        return JFR(V.S(a)[1],C)
    else
        return JFR(V.Σ[1],C)
    end
end

function routine_fNE_mu2(a::Float64, V::Surplus, C::Country2)
    if a<C.A
        return JFR(V.S(a)[2],C)
    else
        return JFR(V.Σ[2],C)
    end
end
      
function integrand_fNE_bef_slow(B::Float64, D::Float64, a::Float64, V::Surplus, C::Country2)
    comp1 = fUI(B,a,V,C)  
    comp2 = C.λ0 / (C.λ0+routine_fNE_mu2(a+B,V,C) )
    integ3 = quadgk(s-> routine_fNE_mu1(a+s,V, C), B, D)[1]
    comp4 = routine_fNE_mu1(a+D, V, C)
    return(comp1 * comp2 * exp(-integ3) * comp4)
end

function integrand_fNE_aft_slow(B::Float64, D::Float64, a::Float64, V::Surplus, C::Country2)
    comp1 = fUI(B,a,V,C)  
    comp2 = C.λ1 / (C.λ1+JFR(V.Σ[3],C) )
    comp3 = exp(-JFR(V.Σ[1],C) * (D-B))
    comp4 = JFR(V.Σ[1],C)
    return(comp1 * comp2 * comp3 * comp4)
end

function fNE_slow(D::Float64, a::Float64, V::Surplus, C::Country2) 
    if a < C.A
        term1 = fUI(D,a,V,C) * routine_fNE_mu2(a+D,V,C) / (C.λ0+routine_fNE_mu2(a+D,V,C))
        term2 = quadgk(B->integrand_fNE_bef_slow(B,D,a,V,C), 0,D)[1] 
        return term1+term2
    else
        term1 = fUI(D,a,V,C) * JFR(V.Σ[3],C) / (C.λ1+JFR(V.Σ[3],C))
        term2 = quadgk(B->integrand_fNE_aft_slow(B,D,a,V,C), 0,D)[1] 
        return term1+term2        
    end
end


function expectedD_slow(a::Float64, V::Surplus, C::Country2)
   # return quadgk(D-> D * fNE_slow(D,a,V,C), 0, Inf, atol=5e-3)[1]
    if a>=C.A
        out = quadgk(D-> D * fNE_slow(D,a,V,C), 0, Inf, atol=5e-3)[1]
        return out
    else
        out = quadgk(D-> D * fNE_slow(D,a,V,C), 0,C.A-a, atol=5e-3)[1]      ## we specify an error of 1e-2 (month)
        out += quadgk(D-> D * fNE_slow(D,a,V,C), C.A-a, Inf, atol=5e-3)[1]
        return out
    end
end

For the life-cycle version, $f_{NE}$ in case 1 is the same as above.

Case 2 for $a<A$ and $a+D> A$:
\begin{align}
f_{NE}(D|a)& =\underbrace{f_{UI}(D|a) . \frac{\mathcal J(\Sigma_2(a+D))}{\lambda_0 + \mathcal J(\Sigma_2(a+D))}}_{\text{term 1}} \notag \\
 & +\mathcal J(\Sigma_1(a+D)) . \int_{0}^{A-a} \underbrace{f_{UI}(B|a) . \frac{\lambda_0 }{\lambda_0 + \mathcal J(S_2 (a+B))}.\exp \left[-\int_{B}^{A-a}\mathcal J(S_1(a+t))dt-\int_{A-a}^{D}\mathcal J(\Sigma_1(a+t))dt \right]}_{\text{integrand_case2A}} dB\notag \\
& +\mathcal J(\Sigma_1(a+D)) . \int_{A-a}^{D} \underbrace{f_{UI}(B|a). \frac{\lambda_0}{\lambda_0 + \mathcal J(\Sigma_2(a+B))} . \exp \left[-\int_{B}^{D}\mathcal J(\Sigma_1(a+t))dt\right]}_{\text{integrand_case2B}} dB.
\end{align}


Case 3 $a\geq A$:
\begin{align}
f_{NE}(D|a)& =\underbrace{f_{UI}(D|a) . \frac{\mathcal J(\Sigma_3(a+D))}{\lambda_1 + \mathcal J(\Sigma_3(a+D))}}_{\text{term 1}} \notag \\
& +\mathcal J(\Sigma_1(a+D)) .\int_{0}^{D} \underbrace{f_{UI}(B|a) . \frac{\lambda_1}{\lambda_1 + \mathcal J(\Sigma_3(a+B))}.\exp \left[
-\int_{B}^{D} \mathcal J(\Sigma_1(a+t))dt\right] }_{\text{integrand_case1}} dB
\end{align}



The expectation is $\mathbb{E}(D|a)=\int_0^{\infty}Df_{NE}(D|a)dD.$

In [ ]:
### case 2
function integrand_fNE_case2A(B::Float64, D::Float64, a::Float64, V::Surplus, C::Country2A)
    comp1 = fUI_case1(B,a,V,C) ## fUI_case1 because a+B<A in case 2A
    comp2 = C.λ0 / (C.λ0+JFR(V.S(a+B)[2],C)) 
    integ3 = quadgk(s-> JFR(V.S(a+s)[1],C) , B, C.A-a)[1]
    integ4 = quadgk(s-> JFR(V.Σ(a+s)[1],C) , C.A-a, D)[1]
    return(comp1 * comp2 * exp(-integ3-integ4))
end

function integrand_fNE_case2B(B::Float64, D::Float64, a::Float64, V::Surplus, C::Country2A)
    comp1 = fUI_case2(B,a,V,C) ## fUI_case2 because A<a+B in case 2B
    comp2 = C.λ0 / (C.λ0+JFR(V.Σ(a+B)[2],C))
    integ3 = quadgk(s-> JFR(V.Σ(a+s)[1],C) , B, D)[1]
    return comp1 * comp2 * exp(-integ3) 
end 

function fNE_case2(D::Float64, a::Float64, V::Surplus, C::Country2A) 
    term1 = fUI_case2(D,a,V,C) * JFR(V.Σ(a+D)[2],C) / (C.λ0+JFR(V.Σ(a+D)[2],C)) ## fUI_case2 because a<A<a+D
    term2A = JFR(V.Σ(a+D)[1],C) * quadgk(B->integrand_fNE_case2A(B,D,a,V,C), 0,C.A-a)[1]
    term2B =  JFR(V.Σ(a+D)[1],C) * quadgk(B->integrand_fNE_case2B(B,D,a,V,C), C.A-a, D)[1]
    return(term1+term2A+term2B)
end
    

### case 3
function integrand_fNE_case3(B::Float64, D::Float64, a::Float64, V::Surplus, C::Country2A)
    comp1 = fUI_case3(B,a,V,C)  ## fUI_case3 because a+B>A
    comp2 = C.λ1 / (C.λ1+JFR(V.Σ(a+B)[3],C))
    integ3 = quadgk(s-> JFR(V.Σ(a+s)[1],C) , B, D)[1]
    return(comp1 * comp2 * exp(-integ3))
end

function fNE_case3(D::Float64, a::Float64, V::Surplus, C::Country2A) 
    term1 = fUI_case3(D,a,V,C) * JFR(V.Σ(a+D)[3],C) / (C.λ1+JFR(V.Σ(a+D)[3],C)) ## fUI_case3 because a+D>A
    term2 = JFR(V.Σ(a+D)[1],C) * quadgk(B->integrand_fNE_case3(B,D,a,V,C), 0,D)[1] 
    return(term1+term2)
end

function expectedD(a::Float64, V::Surplus, C::Country2A)
    D_inf = C.Aret-a ##maximum value of B given retirement age
    if a>=C.A
        out = quadgk(D-> D * fNE_case3(D,a,V,C), 0, D_inf, atol=5e-3)[1]
        return out
    else
        out = quadgk(D-> D * fNE_case1(D,a,V,C), 0,C.A-a, atol=5e-3)[1]      ## we specify an error of 1e-2 (month)
        out += quadgk(D-> D * fNE_case2(D,a,V,C), C.A-a, D_inf, atol=5e-3)[1]
        return out
    end
end

### we code fNE, but this is not necessary
function fNE(D::Float64, a::Float64, V::Surplus, C::Country2A)
    if a+D<C.A
        return(fNE_case1(D, a, V, C))
    elseif a < C.A
        return(fNE_case2(D, a, V, C))
    else
        return(fNE_case3(D, a, V, C)) 
    end
end

### b) Counterfactuals
In a world with unconditional policy, the pdf of $D$ is exactly the one of case 3. In the stationary environment, we have the closed-form solution:
\begin{align}
\mathbb{E}^{without}(D|a)& =\frac{\mathcal J(\Sigma_1)-\mathcal J(\Sigma_3)}{(\lambda_1 +\mathcal J(\Sigma_3))(\mathcal J(\Sigma_1)-\lambda_1 -\mathcal J(\Sigma_3))}  -\frac{\lambda_1}{\mathcal J(\Sigma_1).(\mathcal J(\Sigma_1)-\lambda_1 -\mathcal J(\Sigma_3))}.
\end{align}

In [ ]:
function expectedD_with(a::Float64, V::Surplus, C::Country2) 
    mu1 = JFR(V.Σ[1],C)
    mu3 = JFR(V.Σ[3],C) # with lambdamin
    lam = C.λ1
    return((mu1-mu3)/((lam+mu3)*(mu1-lam-mu3)) - lam/(mu1*(mu1-lam-mu3)))
end
    
fNE_with(D::Float64, a::Float64, V::Surplus, C::Country2A) = fNE_case3(D,a,V,C)

function expectedD_with(a::Float64, V::Surplus, C::Country2A)
    D_inf = C.Aret-a ##maximum value of B given retirement age
    return quadgk(D-> D * fNE_with(D,a,V,C), 0, D_inf, atol=5e-3)[1]
end

In a world without the policy, the pdf of $D$ is 
\begin{align}
f^{without}_{NE}(D|a)& =\underbrace{f^{without}_{UI}(D|a) . \frac{\mathcal J(\Psi_2)}{\lambda_0 + \mathcal J(\Psi_2)}}_{\text{term 1}} \notag \\
& +\mathcal J(\Psi_1) .\int_{0}^{D} \underbrace{f^{without}_{UI}(B|a) . \frac{\lambda_0}{\lambda_0 + \mathcal J(\Psi_2)}.\exp \left[
-\int_{B}^{D} \mathcal J(\Psi_1)dt\right] }_{\text{integrand_case1}} dB
\end{align}
in the stationary environment, which gives a closed form:
\begin{align}
\mathbb{E}(D|a)& =\frac{\mathcal J(\Psi_1)-\mathcal J(\Psi_2)}{(\lambda_0 +\mathcal J(\Psi_2))(\mathcal J(\Psi_1)-\lambda_0 -\mathcal J(\Psi_2))}  -\frac{\lambda_0}{\mathcal J(\Psi_1).(\mathcal J(\Psi_1)-\lambda_0 -\mathcal J(\Psi_2))}.
\end{align}

In the life-cycle envirnoment,
\begin{align}
f^{without}_{NE}(D|a)& =\underbrace{f^{without}_{UI}(D|a) . \frac{\mathcal J(\Psi_2(a+D))}{\lambda_0 + \mathcal J(\Psi_2(a+D))}}_{\text{term 1}} \notag \\
& +\mathcal J(\Psi_1(a+D)) .\int_{0}^{D} \underbrace{f^{without}_{UI}(B|a) . \frac{\lambda_0}{\lambda_0 + \mathcal J(\Psi_2(a+B))}.\exp \left[
-\int_{B}^{D} \mathcal J(\Psi_1(a+t))dt\right] }_{\text{integrand_case1}} dB
\end{align}

In [ ]:
function expectedD_without(a::Float64, V::Surplus, C::Country2) 
    mu1 = JFR(V.Ψ[1],C)
    mu2 = JFR(V.Ψ[2],C) # with lambdamin
    lam = C.λ0
    return (mu1-mu2)/((lam+mu2)*(mu1-lam-mu2)) - lam/(mu1*(mu1-lam-mu2))
end

function integrand_fNE_without(B::Float64, D::Float64, a::Float64, V::Surplus, C::Country2A)
    comp1 = fUI_without(B,a,V,C)  
    comp2 = C.λ0 / (C.λ0+JFR(V.Ψ(a+B)[2],C))
    integ3 = quadgk(s-> JFR(V.Ψ(a+s)[1],C) , B, D)[1]
    return comp1 * comp2 * exp(-integ3)
end

function fNE_without(D::Float64, a::Float64, V::Surplus, C::Country2A) 
    term1 = fUI_without(D,a,V,C) * JFR(V.Ψ(a+D)[2],C) / (C.λ0+JFR(V.Ψ(a+D)[2],C)) 
    term2 = JFR(V.Ψ(a+D)[1],C) * quadgk(B->integrand_fNE_without(B,D,a,V,C), 0,D)[1] 
    return term1+term2 
end

function expectedD_without(a::Float64, V::Surplus, C::Country2A)
    D_inf = C.Aret-a ##maximum value of B given retirement age
    return quadgk(D-> D * fNE_without(D,a,V,C), 0, D_inf, atol=5e-3)[1]
end

## 3) Monte-Carlo simulations
We define a function to compute the average sum of days a worker is nonemployed over 5 years, for workers losing their job at age $A$ (plus). We use a Monte-Carlo method. This is used for Germany only.

In [ ]:
## random generator of duration before job offer from UI
# U is a random term between 0 and 1
draw_jobfinding_fromUI(U::Float64, a::Float64, V::Surplus, C::Country2) = -log(1-U) / JFR(V.Σ[3],C)

cumhaz_jobfinding_fromUI(T::Float64, a::Float64, V::Surplus, C::Country2A) = quadgk( s -> JFR(V.Σ(a + s)[3],C), 0, T)[1]
function draw_jobfinding_fromUI(U::Float64, a::Float64, V::Surplus, C::Country2A) 
    if log(1-U) + cumhaz_jobfinding_fromUI(C.Aret-a,a,V,C) <= 0
        return C.Aret-a
    else
        f = T -> log(1-U) + cumhaz_jobfinding_fromUI(T,a,V,C)
        Df = T -> JFR(V.Σ( a + T)[3],C)
        return find_zero((f,Df), [0, C.Aret-a], Roots.Newton())
    end
end
    
    
## random generator of duration before loss of UI benefit
draw_UIloss(U::Float64, C::Union{Country2,Country2A}) = -log(1-U) / C.λ1


## random generator of duration before job offer from social assistance
draw_jobfinding_from0(U::Float64, a::Float64, V::Surplus, C::Country2) = -log(1-U) / JFR(V.Σ[1],C)

cumhaz_jobfinding_from0(T::Float64, a::Float64, V::Surplus, C::Country2A) = quadgk( s -> JFR(V.Σ( a + s)[1],C), 0, T)[1]
function draw_jobfinding_from0(U::Float64, a::Float64, V::Surplus, C::Country2A) 
    if log(1-U) + cumhaz_jobfinding_from0(C.Aret-a,a,V,C) <= 0
        return C.Aret-a
    else        
        f = T -> log(1-U) + cumhaz_jobfinding_from0(T,a,V,C)
        Df = T -> JFR(V.Σ( a + T)[1],C)
        return find_zero((f,Df), [0, C.Aret-a], Roots.Newton())
    end
end

## random generator of employment duration
draw_jobloss(U::Float64, a::Float64, C::Country2) = -log(1-U) / C.q

cumhaz_jobloss(T::Float64, a::Float64, C::Country2A) = (C.q0+C.qa*(a-C.A)) * T + C.qa * T^2/2
function draw_jobloss(U::Float64, a::Float64, C::Country2A) 
    if C.qa == 0
        return -log(1-U) / C.q0
    else
        AA = C.qa/2
        BB = (C.q0+C.qa*(a-C.A))
        CC = log(1-U)
        if AA * (C.Aret-a)^2 + BB * (C.Aret-a)+ CC <= 0
            return C.Aret-a
        else
            disc = BB^2 - 4 *AA*CC
            return (-BB+sqrt(disc)) / (2*AA)
        end
    end
end

#=cumhaz_jobloss(T::Float64, a::Float64, C::Country2A) = quadgk( s -> qaux( a + s, C), 0, T)[1]
function draw_jobloss(U::Float64, a::Float64, C::Country2A) 
    if log(1-U) + cumhaz_jobloss(C.Aret-a,a,C) <= 0
        return C.Aret-a
    else
        return find_zero(T -> log(1-U) + cumhaz_jobloss(T,a,C), [0, C.Aret-a])
    end
end=#

In [ ]:
function draw_D(rng::MersenneTwister, state_UI::Bool, a::Float64, V::Surplus, C::Union{Country2,Country2A})
    state_E = false
    age = a
    totdur = 0
    while !state_E
        if state_UI
            dur_joboffer = draw_jobfinding_fromUI(rand(rng), age, V, C)
            dur_UIloss = draw_UIloss(rand(rng), C)
            if dur_joboffer < dur_UIloss ##job offer
                dur = dur_joboffer
                age += dur
                totdur += dur
                state_E = true
            else 
                dur = dur_UIloss
                age += dur
                totdur += dur
                state_UI = false
            end
        else
            dur = draw_jobfinding_from0(rand(rng), age, V, C)
            age += dur
            totdur += dur
            state_E = true
        end
    end
    return (totdur, age)
end

### Monte-Carlo version of expectedD
function expectedD_MC(a::Float64, V::Surplus, C::Union{Country2,Country2A}; seed=0, Nind=100000)
    rng = MersenneTwister(seed)
    out = 0
    for i in 1:Nind
        rng_i = MersenneTwister(rand(rng,UInt64))
        out += draw_D(rng_i, true, a, V, C)[1]
    end
    return out/Nind
end

In [ ]:
## random generator of the sum of months as nonemployed over 5 years from age A
function draw_U5(rng::MersenneTwister, V::Surplus, C::Union{Country2,Country2A}) 
    state_E = false
    state_UI = true
    age = C.A
    totdur = 0 ## duration in NE
    elapdur = 0 ## time elapsed
    timeelapsed = false
    while !timeelapsed
        if state_E
            dur = draw_jobloss(rand(rng), age, C)
            elapdur += dur
            if elapdur >= 60
                timeelapsed = true
            else
                age += dur
                state_E = false
                state_UI = rand(rng) < C.χ
            end
        else
            dur, age = draw_D(rng, state_UI, age, V, C)
            elapdur += dur
            if elapdur >= 60
                totdur += dur - (elapdur-60) ###
                timeelapsed = true
            else
                totdur += dur
                state_E = true
            end
        end
    end
    return totdur
end

## Monte-Carlo method to count the expected number of months nonemployed
function average_U5(V::Surplus, C::Union{Country2,Country2A}; seed=0, Nind=100000)
    rng = MersenneTwister(seed)
    out = 0
    for i in 1:Nind
        rng_i = MersenneTwister(rand(rng,UInt64))
        out += draw_U5(rng_i, V, C) 
    end
    return out/Nind
end